In [2]:
import sys, os, pygmt, importlib, re, time
mod_path = '/home/581/da1339/AFIM/src/AFIM/src'
sys.path.insert(0, mod_path)
from sea_ice_toolbox      import SeaIceToolbox
from datetime             import timedelta, date, datetime
from pathlib              import Path
from dask.distributed     import Client, LocalCluster
from dask.diagnostics     import ProgressBar
from pyproj               import CRS, Transformer
from collections          import defaultdict
from scipy.interpolate    import interp1d
from matplotlib.offsetbox import AnchoredOffsetbox, VPacker, TextArea
import numpy                as np
import pandas               as pd
import xarray               as xr
import xesmf                as xe
import matplotlib.pyplot    as plt
import matplotlib.dates     as mdates
import matplotlib.animation as animation
from IPython.display      import Image, HTML, Video

In [1]:
import sys
import importlib
# Step 1: Clear all relevant modules
for mod in list(sys.modules):
    if mod.startswith("sea_ice_toolbox") or mod.startswith("sea_ice_"):
        del sys.modules[mod]
# Step 2: Explicit reloads in dependency order
import sea_ice_plotter
import sea_ice_classification
import sea_ice_icebergs
import sea_ice_observations
import sea_ice_metrics
import sea_ice_toolbox
importlib.reload(sea_ice_plotter)
importlib.reload(sea_ice_classification)
importlib.reload(sea_ice_icebergs)
importlib.reload(sea_ice_observations)
importlib.reload(sea_ice_metrics)
importlib.reload(sea_ice_toolbox)
importlib.reload(pygmt)
# Step 3: Re-import key class from module (to refresh the class definition)
from sea_ice_toolbox import SeaIceToolbox

ModuleNotFoundError: No module named 'sea_ice_plotter'

In [3]:
sim_name   = "elps-min"
dt0_str    = "1994-01-01"
dtN_str    = "1999-12-31"
SI_tools   = SeaIceToolbox(sim_name             = sim_name,
                           dt0_str              = dt0_str,
                           dtN_str              = dtN_str,
                           overwrite_zarr       = True,
                           save_new_figs        = True,
                           show_figs            = True,
                           overwrite_saved_figs = True)
FI_raw, CICE = SI_tools.load_processed_cice( zarr_CICE = True )
FI_bool      = SI_tools.boolean_fast_ice(FI_raw['FI_mask'])
CICE_SO      = CICE.isel(nj=SI_tools.hemisphere_dict['nj_slice'])
hi_SO        = CICE_SO['hi']
aice_SO      = CICE_SO['aice']
tarea_SO     = CICE_SO['tarea']
hi_FI        = hi_SO.where(FI_bool)
aice_FI      = aice_SO.where(FI_bool)
tarea_SO     = tarea_SO.where(FI_bool)

2025-07-10 09:35:58,569 - INFO - log file initialised: /g/data/gv90/da1339/logs/SeaIceToolbox_elps-min.log
2025-07-10 09:36:01,430 - INFO - Initialized new Dask client.
2025-07-10 09:36:01,435 - INFO - Dask distributed client can be accessed at url /proxy/8787/status
2025-07-10 09:36:01,437 - INFO - hemisphere initialised: SH
2025-07-10 09:36:01,453 - INFO - searching for files between 1994-01-01 00:00:00 and 1999-12-31 00:00:00 in /g/data/gv90/da1339/afim_output/elps-min/zarr/ispd_thresh_5.0e-4
2025-07-10 09:36:01,457 - INFO - Found 72 zarr files
2025-07-10 09:37:41,606 - INFO - Loaded FI_BT: 2191 time steps from 72 files
INFO:elps-min:Loaded FI_BT: 2191 time steps from 72 files
2025-07-10 09:37:41,609 - INFO - Memory after Zarr load: 6.8% used
INFO:elps-min:Memory after Zarr load: 6.8% used
2025-07-10 09:37:41,611 - INFO - Load monthly iceh_*.zarr files between 1994-01-01 and 1999-12-31
INFO:elps-min:Load monthly iceh_*.zarr files between 1994-01-01 and 1999-12-31
2025-07-10 09:37:41

In [4]:
sim_name    = "AOM2-ERA5"
SI_tools    = SeaIceToolbox(sim_name            = sim_name,
                           dt0_str              = dt0_str,
                           dtN_str              = dtN_str,
                           overwrite_zarr       = True,
                           save_new_figs        = True,
                           show_figs            = True,
                           overwrite_saved_figs = True)
_,AOM2      = SI_tools.load_processed_cice(zarr_CICE = True)
AOM2_SO     = AOM2.isel(nj=SI_tools.hemisphere_dict['nj_slice'])
hi_AOM2     = AOM2_SO['hi']
aice_AOM2   = AOM2_SO['aice']
tarea_AOM2  = AOM2_SO['tarea']

2025-07-10 09:38:49,404 - INFO - log file initialised: /g/data/gv90/da1339/logs/SeaIceToolbox_AOM2-ERA5.log
INFO:AOM2-ERA5:log file initialised: /g/data/gv90/da1339/logs/SeaIceToolbox_AOM2-ERA5.log
2025-07-10 09:38:49,407 - WARNING - Dask client already exists but was not passed to SeaIceToolbox.
2025-07-10 09:38:49,409 - WARNING - Please explicitly pass the existing Dask client using `client=...` to avoid confusion.
2025-07-10 09:38:49,412 - INFO - Dask distributed client can be accessed at url /proxy/8787/status
INFO:AOM2-ERA5:Dask distributed client can be accessed at url /proxy/8787/status
2025-07-10 09:38:49,415 - INFO - hemisphere initialised: SH
INFO:AOM2-ERA5:hemisphere initialised: SH
2025-07-10 09:38:49,443 - INFO - searching for files between 1994-01-01 00:00:00 and 1999-12-31 00:00:00 in /g/data/gv90/da1339/afim_output/AOM2-ERA5/zarr/ispd_thresh_5.0e-4
INFO:AOM2-ERA5:searching for files between 1994-01-01 00:00:00 and 1999-12-31 00:00:00 in /g/data/gv90/da1339/afim_output/A

In [5]:
P_ESA_CCI_reG           = Path(Path.home(),"seaice","ESA_CCI","L2P","envisat","sh","reG","ESA_CCI_SIT_regridded.zarr")
ESA_CCI                 = xr.open_zarr(P_ESA_CCI_reG)
dt                      = pd.to_datetime(ESA_CCI["time"].values)
ESA_CCI.coords['doy']   = ('time', dt.dayofyear)
ESA_CCI['ESA_sit_clim'] = ESA_CCI['ESA_sit'].groupby('doy').mean('time')
ESA_valid_mask          = ~xr.ufuncs.isnan(ESA_CCI['ESA_sit_clim'].isel(doy=0))  # or use `.isel(doy=slice(None)).any('doy')`

In [ ]:
def compute_model_climatology(sit):
    return sit.assign_coords(doy=sit['time'].dt.dayofyear).groupby('doy').mean('time', skipna=True)
def compute_volume_climatology(sic, sit, area):
    return SI_tools.compute_ice_volume( sic, sit, area ).groubby('doy')
def spatially_integrated_sit(clim, valid_mask):
    return clim.where(valid_mask).mean(dim=['nj', 'ni'], skipna=True)

In [ ]:
sit_CICE_clim = compute_model_climatology(hi_SO)
sit_FI_clim   = compute_model_climatology(hi_FI)
sit_AOM2_clim = compute_model_climatology(hi_AOM2)
siv_CICE_clim = compute_volume_climatology(aice_SO, hi_SO, tarea_SO)
siv_FI_clim   = compute_volume_climatology(aice_FI, hi_FI, tarea_FI)
siv_AOM2_clim = compute_volume_climatology(aice_AOM2, hi_AOM2, tarea_AOM2)

In [ ]:
esa_daily_mean     = spatially_integrated_sit(ESA_CCI['ESA_sit_clim'], ESA_valid_mask)
cice_daily_mean    = spatially_integrated_sit(sit_CICE_clim, ESA_valid_mask)
fast_daily_mean    = spatially_integrated_sit(sit_FI_clim, ESA_valid_mask)
aom2_daily_mean    = spatially_integrated_sit(sit_AOM2_clim, ESA_valid_mask)

In [ ]:
plt.figure(figsize=(18,10))
doy = esa_daily_mean['doy']
plt.plot(doy, esa_daily_mean, label='ESA-CCI (2002-2012)', color="black")
plt.plot(doy, cice_daily_mean, label='CICE6-SA (elps-min) all sea ice (1994-1999)')
plt.plot(doy, fast_daily_mean, label='CICE6-SA (elps-min) fast ice only (1994-1999)')
plt.plot(doy, aom2_daily_mean, label='ACCESS-OM2 (ERA5-forced) all sea ice (1994-1999)')
plt.xlabel('Day of Year', fontsize=12)
plt.xlim([5,360])
plt.ylabel('Sea Ice Thickness (m)', fontsize=12)
plt.ylim([0,5])
plt.legend()
plt.tight_layout()
plt.savefig(Path(SI_tools.D_graph, sim_name, "SIT_vs_ESA_CCI_climatology.png"), dpi=150)
plt.show()

In [ ]:
sim_name = "elps-min"
dt0_str  = "1994-01-01"
dtN_str  = "1999-12-31"
SI_tools = SeaIceToolbox(sim_name             = sim_name,
                         dt0_str              = dt0_str,
                         dtN_str              = dtN_str,
                         overwrite_zarr       = True,
                         save_new_figs        = True,
                         show_figs            = True,
                         overwrite_saved_figs = True)
P_met    = Path(SI_tools.D_sim,"zarr","ispd_thresh_5.0e-4","metrics","FI_BT_bool_mets.zarr")
FI_met   = xr.open_zarr(P_met)
FI_met['FIV'].plot()
# FI_BT_bool = xr.open_zarr( "/g/data/gv90/da1339/afim_output/ndte-max-re-off/zarr/ispd_thresh_5.0e-4/metrics/FI_BT_bool_mets.zarr" )
# AF2020_CSV = SI_tools.load_AF2020_FIA_summary( start=dt0_str, end=dtN_str )
# FIA_obs    = AF2020_CSV['FIA_clim_repeat'].sel(region='circumpolar')
# # print(FI_BT_bool['FIA'])
# # print(FIA_obs)
# P_png = Path(SI_tools.D_graph, sim_name, f"FIA_FIP_{sim_name}_{SI_tools.ispd_thresh_str}_1994-1999.png")
# #SI_tools.pygmt_map_plot_one_var(FI_BT_bool['FIP'], 'FIP', plot_regions=2, plot_bathymetry=False, plot_GI=True if SI_tools.use_gi else False)
# SI_tools.plot_FIA_FIP_faceted( {'FI_BT_bool':FI_BT_bool['FIA']}, FI_BT_bool['FIP'], P_png=P_png, plot_GI=True if SI_tools.use_gi else False)

In [62]:
sim_name = "elps-min"
#for region in SI_tools.Ant_8sectors.keys():
SI_tools = SeaIceToolbox(sim_name=sim_name)
D_png = Path(SI_tools.D_graph, sim_name, "south", "hi")
D_ani = Path(SI_tools.D_graph, "animations", sim_name, "hi")
F_ani = f"{sim_name}_hi_south.mp4"
P_ani = Path(D_ani,F_ani)
P_mp4 = Path.home() / "AFIM" / "src" / "AFIM" / "docs" / "figures" / F_ani
D_ani.mkdir(parents=True, exist_ok=True)
frames = sorted([f for f in os.listdir(D_png) if f.endswith(".png")])
os.system(f"rm {SI_tools.D_tmp}/frame_*.png")
for i, f in enumerate(frames):
    src = D_png / f
    dst = Path(SI_tools.D_tmp) / f"frame_{i:04d}.png"
    if not dst.exists():
        os.symlink(src, dst)
os.system(f"ffmpeg -y -r 2 -i {SI_tools.D_tmp}/frame_%04d.png -vf \"scale=iw:ih+mod(2-ih\\,2)\" -c:v libx264 -pix_fmt yuv420p {P_ani}")
os.system(f"cp {P_ani} {P_mp4}")

2025-07-10 11:41:29,773 - WARNING - Dask client already exists but was not passed to SeaIceToolbox.
2025-07-10 11:41:29,775 - WARNING - Please explicitly pass the existing Dask client using `client=...` to avoid confusion.
2025-07-10 11:41:29,778 - INFO - Dask distributed client can be accessed at url /proxy/8787/status
INFO:elps-min:Dask distributed client can be accessed at url /proxy/8787/status
2025-07-10 11:41:29,780 - INFO - hemisphere initialised: SH
INFO:elps-min:hemisphere initialised: SH
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/g/data/xp65/public/./apps/med_conda/envs/analysis3-25.05 --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1732155191655/_build

0

In [ ]:
CICE_SO     = CICE.isel(nj=SI_tools.hemisphere_dict['nj_slice'])
sim_name    = "AOM2-ERA5"
dt0_str     = "1993-01-01"
dtN_str     = "1999-12-31"
SI_tools    = SeaIceToolbox(sim_name             = sim_name,
                           dt0_str              = dt0_str,
                           dtN_str              = dtN_str,
                           overwrite_zarr       = True,
                           save_new_figs        = True,
                           show_figs            = True,
                           overwrite_saved_figs = True)
DS, AOM2    = SI_tools.load_processed_cice(zarr_CICE = True)
AOM2_SO     = AOM2.isel(nj=SI_tools.hemisphere_dict['nj_slice'])
OSI_SAF     = xr.open_mfdataset("/home/581/da1339/seaice/OSI_SAF/ispd_reG_SH*")

In [ ]:
AOM2_ispd   = SI_tools.compute_ice_magnitude_from_ice_components_on_Bgrid(AOM2, ivec_type = 'BT')
CICE_ispd   = SI_tools.compute_ice_magnitude_from_ice_components_on_Bgrid(CICE, ivec_type = 'BT')

In [ ]:
from PIL import Image
def stitch_fip_side_by_side(sim_model, sim_obs, base_dir, region_list, output_path, dpi=200):
    nrows = len(region_list)
    fig, axs = plt.subplots(nrows=nrows, ncols=3, figsize=(20,60))
    for row, region in enumerate(region_list):
        for col, sim in enumerate([sim_obs, sim_model, "FIP_delta"]):  # left: obs, right: model
            if col==2:
                path = os.path.join(base_dir, sim_model, region, "FIP_delta", f"2000-2018_{sim_model}_{region}_FIP_delta.png")
            else:
                path = os.path.join(base_dir, sim, region, "FIP", f"2000-2018_{sim}_{region}_FIP.png")
            if not os.path.exists(path):
                print(f"❌ Missing: {path}")
                axs[row, col].axis("off")
                continue
            img = Image.open(path)
            axs[row, col].imshow(img)
            axs[row, col].axis("off")
    axs[0, 0].set_title("AF2020 (obs)"       , fontsize=14, fontweight="bold")
    axs[0, 1].set_title(f"{sim_model} (sim)" , fontsize=14, fontweight="bold")
    axs[0, 2].set_title("Difference: obs-sim", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.savefig(output_path, dpi=dpi)
    plt.close()
    print(f"✅ Saved stitched figure: {output_path}")
region_list = ["DML", "WIO", "EIO", "Aus", "VOL", "AS", "BS", "WS"]
stitch_fip_side_by_side("elps-min", "AF20",
                  base_dir="/g/data/gv90/da1339/GRAPHICAL/AFIM",
                  region_list=region_list,
                  output_path="/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/FIP_panel_AF20_vs_elps-min.png")